In [1]:
!pwd

/home/techie/Desktop/general/end-to-end-liver-project/research


In [2]:
import os

os.chdir("../")
!pwd

/home/techie/Desktop/general/end-to-end-liver-project


In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from liver.constants import *
from liver.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self, config_file= CONFIG_FILE_PATH):
        self.config_path = read_yaml(config_file)
        
        create_directories([self.config_path.artifacts_root])
        
    
    def dataIngestionConfig(self) -> DataIngestionConfig:
        config = self.config_path.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )   
        
        return data_ingestion_config     

In [6]:
import os
import urllib.request as request
from liver import logger
from liver.utils.common import get_size
import zipfile

In [9]:
class DataIngestion:
    def __init__(self, confg: DataIngestionConfig):
        self.config = confg
        
    
    def download_dataset(self):
        
        if not os.path.exists(self.config.local_data_file):
            filename, headers= request.urlretrieve(
                                    url= self.config.source_URL,
                                    filename=self.config.local_data_file
                                )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"file already exist, size--> {get_size(Path(self.config.local_data_file))}")
    
    
    def extract_zip(self):
        
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.dataIngestionConfig()
    data_ingestinion = DataIngestion(confg=data_ingestion_config)
    data_ingestinion.download_dataset()
    data_ingestinion.extract_zip()
except Exception as e:
    raise e

[2023-10-19 18:09:28,574: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-19 18:09:28,578: INFO: common: created directory at: artifacts]
[2023-10-19 18:09:28,582: INFO: common: created directory at: artifacts/data_ingestion]
[2023-10-19 18:09:30,391: INFO: 1375348243: artifacts/data_ingestion/liver.zip download! with following info: 
Connection: close
Content-Length: 7702
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "4247b84e56b3a8a30ec50dffd29342df15c3d56bc81a020fbeb426b6315c3b11"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 0C28:1226E:A07F1F:A68790:65315FBD
Accept-Ranges: bytes
Date: Thu, 19 Oct 2023 17:09:30 GMT
Via: 1.1 varnish
X-Served-By: cache-lon4222-LON
X-Cache: HIT
X-Cache-Hits: 1
X-Timer: S1697735370.342284,VS0,VE1
Vary: Authorization,Accept-Encodi